In [6]:
import numpy as np
# import pandas as pd

In [7]:
def objective_function(location):
    return np.sum(location**2)

def denorm(data, domain):
    domain.sort()
    return data * (domain[:, 1] - domain[:, 0]) + domain[:, 0] # data * (max - min) + min

In [8]:
class Property:
    def __init__(self, location, objective_function, domain, price=None, demand=None, supply=None):
        self.location = location
        self.objective_function = objective_function
        self.domain = domain
        self.price = price
        self.demand = demand
        self.supply = supply

    def value(self):
        return self.objective_function(denorm(self.location, self.domain))

In [9]:
class Supplier:
    def __int__(self, current_location):
        self.current_location = current_location

    def  move(self, ideal_property, max_value, min_value, n_supplier, k_sigma_s):
        norm_value = (ideal_property.value  - min_value)/(max_value - min_value)
        q_supply = (1 - norm_value)/(n_supplier + 1)
        # next_location_mean = self.next_location.location
        # next_location_std = k_sigma_s * abs(self.next_location.price) * np.sqrt(-2*np.log(self.purchase_power))
        new_location = np.copy(np.random.normal(ideal_property.location, k_sigma_s*q_supply))
        new_location[new_location>1] = 1
        new_location[new_location<0] = 0
        self.current_location.location = np.copy(new_location)

In [10]:
class Demander:
    def __init__(self, current_location, objective_function,
                 friend=None, best_location=None, next_location=None, purchase_power=None):
        self.current_location = current_location
        if best_location is None:
            self.best_location = Property(np.copy(current_location.location), current_location.objective_function, current_location.domain)
        else:
            self.best_location = best_location
        if next_location is None:
            self.next_location = Property(np.copy(current_location.location), current_location.objective_function, current_location.domain)
        else:
            self.next_location = next_location
        self.objective_function = objective_function
        if friend is None:
            self.friend = []
        else:
            self.friend = friend
        if purchase_power is None:
            self.purchase_power = np.random.rand()

    def add_friend(self, demander_friend):
        self.friend.append(demander_friend)

    def move(self):
        self.current_location = self.next_location
        if self.next_location.value() > self.best_location.value():
            self.best_location = self.current_location

    def communication(self, k_sigma_d):
        self.next_location = Property(location=np.copy(self.best_location.location), objective_function=self.best_location.objective_function, domain=self.best_location.domain, price=self.best_location.price)

        for i in self.friend:
            if i.best_location.value > self.best_location.value:
                self.next_location = Property(np.copy(i.best_location.location), i.best_location.objective_function, i.best_location.domain)

        next_location_mean = self.next_location.location
        next_location_std = k_sigma_d * abs(self.next_location.price) * np.sqrt(-2*np.log(self.purchase_power))

        new_location = np.random.normal(next_location_mean, next_location_std)
        new_location[new_location>1] = 1
        new_location[new_location<0] = 0

        self.next_location = Property(np.copy(new_location), i.best_location.objective_function, i.best_location.domain)

In [13]:
class Market:
    def __init__(self, objective_function, domain, demander_number, supplier_number, max_friends, characteristics_length, k_sigma_d, k_sigma_s, k_num_s, demanders=None, suppliers = None):
        self.max_friends = max_friends
        dimension = len(domain)
        if demanders is None:
            self.demanders = [Demander(Property(np.random.uniform(size=(dimension,)), objective_function, domain), objective_function) for _ in range(demander_number)]
        else:
            self.demanders = demanders
        if suppliers is None:
            self.suppliers = [suppliers(Property(np.random.uniform(size=(dimension,)), objective_function, domain)) for _ in range(demander_number)]
        else:
            self.suppliers = suppliers
        self.characteristics_length = characteristics_length
        self.k_sigma_d = k_sigma_d
        self.k_num_s = k_num_s
        self.k_sigma_s = k_sigma_s

    def make_friend(self):
        for i in self.demanders:
            i.friend = []
            [i.add_friend(np.copy(j)) for j in self.demanders[np.random.choice(len(self.demanders), size=self.max_friends, replace=False)]]

    def demand_evaluation(self):
        occupied_property = []
        [occupied_property.append(np.copy(i.current_location.location)) for i in self.demanders]
        for i in self.demanders:
            i.current_location.demand = (1/self.characteristics_length/len(self.demanders)) * sum(self.characteristics_length - np.sqrt(np.sum((i.current_location.loc - occupied_property)**2)))
            i.best_location.demand = np.copy(i.current_location.demand) #### bug in sum(self.characteristics_length - np.sqrt(np.sum((i.current_location.loc - occupied_property)**2))) calculation dont have bug but check in debug

    def supply_evaluation(self):
        occupied_property = []
        [occupied_property.append(np.copy(i.current_location.location)) for i in self.suppliers]
        for i in self.demanders:
            i.current_location.demand = (1/self.characteristics_length/len(self.suppliers)) * sum(self.characteristics_length - np.sqrt(np.sum((i.current_location.loc - occupied_property)**2)))
            i.best_location.demand = np.copy(i.current_location.demand)

    def price_evaluation(self):
        self.demand_evaluation()
        self.supply_evaluation()
        for i in self.demanders:
            i.current_location.price = i.current_location.demand - i.current_location.supply
            i.best_location.price = i.best_location.demand - i.best_location.supply

    def demander_update(self):
        for i in self.demanders:
            i.communication(self.k_sigma_d)
            i.move()

    def supplier_update(self):
        n_supplier = np.ceil(self.k_num_s*len(self.suppliers))
        # selected = randi(length(obj.spl), 1, nSupplier);
        # selected_supplier = np.array([np.copy(i) for i in self.suppliers[np.random.choice(len(self.suppliers), size=self.max_friends, replace=False)]])
        max_value = -np.inf
        max_price = -np.inf
        min_value =  np.inf

        for i in self.demanders:
            if i.best_location.value > max_value:
                max_value = np.copy(i.best_location.value)
            if i.current_location.value > max_value:
                max_value = np.copy(i.current_location.value)
            if i.best_location.value < min_value:
                min_value = np.copy(i.best_location.value)
            if i.current_location.value < min_value:
                min_value = np.copy(i.current_location.value)
            if i.current_location.price > max_price:
                max_price = np.copy(i.current_location.price)
                ideal_property = i.current_location
            if i.best_location.price > max_price:
                max_price = np.copy(i.best_location.price)
                ideal_property = i.best_location


        [i.move(ideal_property, max_value, min_value, n_supplier, self.k_sigma_s) for i in self.suppliers[np.random.choice(len(self.suppliers), size=self.max_friends, replace=False)]]

In [14]:
location = np.random.uniform(size=(2,))

domain = np.array([[-65, 65], [-65, 65]])

p_1 = Property(location, objective_function, domain)
 # objective_function, domain, demander_number, supplier_number, max_friends, characteristics_length, k_sigma_d, k_sigma_s, k_num_s, demanders=None, suppliers = None):
market = Market(objective_function=objective_function, domain=domain, demander_number=80, supplier_number=20, max_friends=20, characteristics_length=np.sqrt(len(domain)), k_sigma_d=0.7, k_sigma_s=0.7, k_num_s=0.4)
#
# (self, objective_function, domain, demander_number, supplier_number, max_friends, characteristics_length, k_sigma_d, k_sigma_s, k_num_s, demanders=None, suppliers = None):

TypeError: 'NoneType' object is not callable